# Предсказание цен машин с Юлы




In [53]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, Adadelta # Алгоритмы оптимизации, для настройки скорости обучения
import numpy as np # Библиотека работы с массивами
import matplotlib.pyplot as plt
import pandas as pd # Библиотека pandas
from sklearn.preprocessing import LabelEncoder, StandardScaler # Функции для нормализации данных
from sklearn import preprocessing
from tensorflow import keras
from tensorflow.keras.metrics import MeanAbsoluteError, MeanSquaredError, R2Score

In [25]:
# Считываем csv с помощью pandas
cars = pd.read_csv('/content/cars_new.csv', sep=',')
cars[:5] # Выводим первые 5 машин

,mark,model,price,year,mileage,body,kpp,fuel,volume,power
0,kia,cerato,996000,2018,28000,седан,автомат,бензин,2.0,150.0
1,daewoo,nexia 1 поколение [2-й рестайлинг],140200,2012,60500,седан,механика,бензин,1.5,80.0
2,suzuki,jimny 3 поколение [рестайлинг],750000,2011,29000,внедорожник,автомат,бензин,1.3,85.0
3,bmw,x1 18 e84 [рестайлинг],970000,2014,49500,кроссовер,автомат,бензин,2.0,150.0
4,chevrolet,lacetti 1 поколение,205000,2007,151445,седан,механика,бензин,1.4,95.0


In [26]:
print(cars.values.shape)

(70119, 10)


In [27]:
# Создаём словарь поле - его индекс
def create_dict(s):
  ret = {}                          # Создаём пустой словарь
  for _id, name in enumerate(s):    # Проходим по всем парам - id и название
    ret.update({name: _id})         # Добавляем в словарь
  return ret

# Функция преобразования в one hot encoding
def to_ohe(value, d):
  arr = [0] * len(d)
  arr[d[value]] = 1
  return arr

In [28]:
# Создаём словари по всем текстовым колонкам
marks_dict = create_dict(set(cars['mark']))
models_dict = create_dict(set(cars['model']))
bodies_dict = create_dict(set(cars['body']))
kpps_dict = create_dict(set(cars['kpp']))
fuels_dict = create_dict(set(cars['fuel']))

# Запоминаем цены
prices = np.array(cars['price'], dtype=np.float)

# Запоминаем числовые параметры
# и нормируем
years = preprocessing.scale(cars['year'])
mileages = preprocessing.scale(cars['mileage'])
volumes = preprocessing.scale(cars['volume'])
powers = preprocessing.scale(cars['power'])

<ipython-input-28-006031011196>:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  prices = np.array(cars['price'], dtype=np.float)


In [29]:
print(fuels_dict)

{'гибрид': 0, 'дизель': 1, 'газ': 2, 'бензин': 3, 'электро': 4}


In [30]:
volumes.mean()

1.2879530757014324e-16

In [31]:
volumes.std()

1.0

In [32]:
print(powers)

[ 0.22905575 -0.95630346 -0.87163494 ...  0.22905575 -0.75309902
  2.34576862]


In [33]:
# Создаём пустую обучающую выборку
x_train = []
y_train = []

# Проходам по всем машинам
for _id, car in enumerate(np.array(cars)):
  # В y_train добавляем цену
  y_train.append(prices[_id])

  # В x_train объединяем все параметры
  # Категорийные параметры добавляем в виде ohe
  # Числовые параметры добавляем напрямую
  x_tr = to_ohe(car[0], marks_dict) + \
        to_ohe(car[1], models_dict) + \
        to_ohe(car[5], bodies_dict) + \
        to_ohe(car[6], kpps_dict) + \
        to_ohe(car[7], fuels_dict) + \
        [years[_id]] + \
        [mileages[_id]] + \
        [volumes[_id]] + \
        [powers[_id]]

  # Добавляем текущую строку в общий x_train
  x_train.append(x_tr)

# Превращаем лист в numpy.array
x_train = np.array(x_train, dtype=np.float)
y_train = np.array(y_train, dtype=np.float)

<ipython-input-33-56e6415ba6d7>:27: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x_train = np.array(x_train, dtype=np.float)
<ipython-input-33-56e6415ba6d7>:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_train = np.array(y_train, dtype=np.float)


In [34]:
# Выводим размеры выборок
print(x_train.shape)
print(y_train.shape)

(70119, 3208)
(70119,)


In [35]:
# Выводим один x_train
print(x_train[0,:20])
print(x_train[0,-20:])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          1.          0.          0.
  0.          0.          1.          0.          1.5200145  -1.40018212
  0.12288486  0.22905575]


In [36]:
# Выводим y_train
print(y_train[:10])

[ 996000.  140200.  750000.  970000.  205000.  985000.  589000.  500000.
 1320000.  270000.]


In [37]:
y_train.shape

(70119,)

In [38]:
y_train.reshape(-1,1).shape

(70119, 1)

In [39]:
# Нормализуем y_train
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1,1)).flatten()

# y_train.reshape(-1,1) добавляет одну размерность
# Это нужно потому, что y_scaler.fit_transform
# Требует двумерны вектор, массив примеров, которые надо нормализовать
# Он не умеет работать с одним примеров
# Поэтому мы делаем массив из одного примера
# На выходе он так же выдаёт массив примеров
# Но нам нужен только первый пример
# Поэтому мы делаем flatten() - уменьшение размерности

In [40]:
y_train_scaled.shape

(70119,)

In [41]:
# Выводим базовый и нормированный y_train
print(y_train[:10])
print(y_train_scaled[:10])

[ 996000.  140200.  750000.  970000.  205000.  985000.  589000.  500000.
 1320000.  270000.]
[ 0.75082432 -0.62135434  0.3563911   0.70913625 -0.51745486  0.73318706
  0.09824578 -0.04445567  1.27032173 -0.4132347 ]


In [44]:
X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [48]:
model_ula = keras.Sequential()

In [49]:
model_ula.add(Dense(3208, activation='relu'))
model_ula.add(Dropout(0.3))

In [50]:
model_ula.add(Dense(30, activation='relu'))
model_ula.add(Dropout(0.3))


In [51]:
model_ula.add(Dense(1, activation='linear'))

In [56]:
model_ula.compile(optimizer='adam', loss='mse', metrics=[R2Score()])


In [58]:
model_ula.fit(X_train, y_train, epochs=100, verbose=1, batch_size=50, validation_split=0.2)

Epoch 1/100
898/898 [==============================] - 100s 111ms/step - loss: 664555880448.0000 - r2_score: -0.7268 - val_loss: 671517376512.0000 - val_r2_score: -0.6901
Epoch 2/100
898/898 [==============================] - 96s 107ms/step - loss: 639566544896.0000 - r2_score: -0.6619 - val_loss: 560367337472.0000 - val_r2_score: -0.4104
Epoch 3/100
898/898 [==============================] - 96s 107ms/step - loss: 437375467520.0000 - r2_score: -0.1365 - val_loss: 315265286144.0000 - val_r2_score: 0.2065
Epoch 4/100
898/898 [==============================] - 98s 109ms/step - loss: 335571124224.0000 - r2_score: 0.1280 - val_loss: 247909564416.0000 - val_r2_score: 0.3760
Epoch 5/100
898/898 [==============================] - 97s 108ms/step - loss: 312110907392.0000 - r2_score: 0.1890 - val_loss: 225934491648.0000 - val_r2_score: 0.4313
Epoch 6/100
898/898 [==============================] - 106s 118ms/step - loss: 306534973440.0000 - r2_score: 0.2035 - val_loss: 214331146240.0000 - val_r2

In [59]:
y_pred = model_ula.predict(X_test)

439/439 [==============================] - 5s 12ms/step
